# Querying Multiple Data Formats 
*By Winston Robson, edited for Dask-SQL by Shondace Thomas*

In this notebook, we will cover: 
- How to create DaskSQL tables from: 
    - Text file (CSV)
    - Apache Parquet 
    - cuDF DataFrame (GDF)
- How to `JOIN` multiple DaskSQL tables into one cuDF DataFrame with a single query.

#### Download Data
This cell will check if you have the data for this demo, and, if you don't, will download it for you.

In [1]:
import os

# relative path to data folder
data_dir = '../../../data/dask-sql/'

# does folder exist?
if not os.path.exists(data_dir):
    print('creating dask-sql directory\n')
    # create folder
    os.system('mkdir ../../data/dask-sql')

# do we have file 0?
if not os.path.isfile(data_dir + 'cancer_data_00.csv'):
    !wget -P ../../../data/dask-sql https://raw.githubusercontent.com/BlazingDB/bsql-demos/master/data/cancer_data_00.csv

# do we have file 1?
if not os.path.isfile(data_dir + 'cancer_data_01.parquet'):
    !wget -P ../../../data/dask-sql https://blazingsql-colab.s3.amazonaws.com/cancer_data/cancer_data_01.parquet

# do we have file 2?
if not os.path.isfile(data_dir + 'cancer_data_02.csv'):
    !wget -P ../../../data/dask-sql https://raw.githubusercontent.com/BlazingDB/bsql-demos/master/data/cancer_data_02.csv

## Create DaskContext
You can think of the DaskContext much like a Spark Context (i.e. information such as FileSystems registered & Tables created will be stored).

In [2]:
from dask_sql import Context
# start up DaskSQL
dc = Context()

#### Data Path
Dask-SQL requires the full path to the data. This cell uses the `pwd` bash command to identify the path to this directory, then add it to the relative path to the notebooks-contrib `data` directory (i.e. what you'd type in Terminal to navigate to the data).

In [3]:
# bash command, returns SList w/ path (str) at 0th index
path = !pwd

# extract path to notebooks-contrib
path = path[0].split('getting_started_materials')[0] 

# add path to Dask-SQL data
path = path + 'data/dask-sql/'

# what's it look like?
path

'/rapids/notebooks/extra/notebooks-contrib/data/dask-sql/'

### Create Table from CSV
Here we create a Dask-SQL table directly from a comma-separated values (CSV) file. 

In [4]:
# define column names and types
col_names = ['diagnosis_result', 'radius', 'texture', 'perimeter']
col_types = ['float32', 'float32', 'float32', 'float32'] 

# create table from CSV file
dc.create_table('data_00', path +'cancer_data_00.csv', gpu=True, names=col_names, dtype=col_types)


# df_result = dc.sql("SELECT * FROM data_00")
# df_result.head()
# type(df_result)
# df_result

### Create Table from Parquet
Here we create a Dask-SQL table directly from an Apache Parquet file.

In [5]:
# create table from Parquet file
dc.create_table('data_01', path+'cancer_data_01.parquet', gpu= True)

### Create Table from GPU DataFrame
Here we use cuDF to create a GPU DataFrame (GDF), then use Dask-SQL to create a table from that GDF.

The GDF is the standard memory representation for the RAPIDS AI ecosystem.

In [7]:
import cudf

# define column names & types
col_names = ['compactness', 'symmetry', 'fractal_dimension']
col_types = ['float32', 'float32', 'float32']

# make GPU DataFrame from CSV w/ cuDF
gdf_02 = cudf.read_csv(path+'cancer_data_02.csv', names=col_names, dtype=col_types)

# create BlazingSQL table from cuDF DataFrame
dc.create_table('data_02', gdf_02)

In [8]:
import time

t0 = time.time()

# Join Tables Together 

Now we can use Dask-SQL to join all three data formats in a single federated query. Dask-SQL queries return results as a cuDF DataFrame.

In [9]:
# grab everything from data_00 & data_01 and area & smoothness from data_01
query = '''
        SELECT 
            a.*, 
            b.area, b.smoothness, 
            c.* 
        FROM 
            data_00 AS a
        LEFT JOIN data_01 AS b
            ON (a.perimeter = b.perimeter)
        LEFT JOIN data_02 AS c
            ON (b.compactness = c.compactness)
            '''

# join the tables together
join = dc.sql(query)

# display results (type(join)==cudf.core.dataframe.DataFrame)
join.head()

,diagnosis_result,radius,texture,perimeter,area,smoothness,compactness,symmetry,fractal_dimension
0,0.0,17.0,21.0,81.0,503.0,0.098,0.052,0.159,0.057
1,1.0,21.0,18.0,124.0,1076.0,0.110,0.169,0.191,0.060
2,0.0,9.0,26.0,59.0,251.0,0.107,0.141,0.211,0.080
3,0.0,9.0,26.0,59.0,261.0,0.077,0.088,0.234,0.070
4,1.0,22.0,25.0,90.0,584.0,0.101,0.128,0.166,0.066


In [10]:
t1 = time.time()
print(f"run_stuff took {t1-t0}s")

run_stuff took 3.951575756072998s


In [11]:
len(join.index)

316

# You're Ready to Rock
And... thats it! You are now live with Dask-SQL.

Check out our [docs](https://dask-sql.readthedocs.io/) to get fancy or to learn more about how Dask-SQL works with the rest of [RAPIDS AI](https://rapids.ai/).